In [18]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt
import urllib3
# import re2
import regex
import re
import io

In [20]:
url = 'https://www.opensecrets.org/races/summary.csv?cycle=2020&id=TN07'

r = requests.get(url)
data = r.content.decode('utf8')
df = pd.read_csv(io.StringIO(data))

df

,cid,FirstLastP,Rcpts,Spent,PACs,Indivs,Cand,Other,EndCash,LgIndivs,...,Result,CRPICO,State,IncCID,Incumbent,primarydate,DistIDCurr,capeye,sort,SmLgIndivsNote
0,N00041873,Mark Green (R),1194960.47,935486.67,171900.0,819151.42,0.0,203909.05,287888.55,819151.42,...,W,I,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN07,0,1,N
1,N00045536,Kiran Sreepada (D),206644.28,207190.98,4000.0,202644.28,0.0,0.00,0.00,179129.75,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,,0,2,N
2,N00047077,Ronald Brown (I),1750.00,0.00,0.0,1750.00,0.0,0.00,9006.00,300.00,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,,0,2,N
3,N00046592,Scott Vieira Jr (I),655.47,1048.51,10.0,45.00,35.0,565.47,-196.52,0.00,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,,0,2,N
4,N00045535,Benjamin Estes (3),0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,...,,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,NaN,0,2,N


<bound method Response.json of <Response [200]>>